# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time
from matplotlib import pyplot as plt

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_py = pd.read_csv("Weather Output Data/cities_output.csv")
weather_py.head(10)

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Mataura,92,NZ,1628449212,99,-46.1927,168.8643,35.47,3.33
1,Bredasdorp,100,ZA,1628449083,58,-34.5322,20.0403,58.32,13.80
2,Provideniya,100,RU,1628448853,81,64.3833,-173.3000,45.39,15.93
3,Dikson,0,RU,1628449354,79,73.5069,80.5464,46.08,5.19
4,Englewood,0,US,1628449213,21,39.6478,-104.9878,93.40,3.00
5,Hobart,12,AU,1628448932,88,-42.8794,147.3294,42.66,1.99
6,Sabang,100,ID,1628449430,77,5.8933,95.3214,82.99,14.18
7,Ternate,100,ID,1628449133,81,0.8000,127.4000,79.12,1.43
8,Saint George,1,US,1628449348,16,37.1041,-113.5841,102.67,1.99
9,Oktyabr'skoye,7,RU,1628449431,94,43.0645,44.7417,68.14,4.47


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)
locations = weather_py[["Lat", "Lng"]]
humidity = weather_py["Humidity"]

In [4]:
#Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=1)
fig.add_layer(heat_layer)

#Display
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
city_conditions = weather_py.loc[(weather_py["Wind Speed"] <= 10) & (weather_py["Cloudiness"] == 0) & \
                                   (weather_py["Max Temp"] >= 70) & (weather_py["Max Temp"] <= 80)].dropna()

city_conditions

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
108,Hambantota,0,LK,1628449298,94,6.1241,81.1185,79.14,6.91
177,Zhangye,0,CN,1628449182,33,38.9342,100.4517,70.47,4.76
188,Libohovë,0,AL,1628449470,44,40.0311,20.2631,77.88,1.48
189,Lata,0,PT,1628449470,59,40.1629,-8.3327,74.05,6.67
195,Erzin,0,TR,1628449254,71,36.9559,36.2000,79.32,2.17
207,Estrela,0,BR,1628449474,41,-28.0611,-50.9383,72.54,1.68
216,Mtimbira,0,TZ,1628449476,53,-8.7833,36.3500,70.34,2.33
223,Caravelas,0,BR,1628449479,68,-17.7125,-39.2481,73.65,8.34


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Create a hotel_df
hotel_df = city_conditions.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel Name
108,Hambantota,LK,6.1241,81.1185,
177,Zhangye,CN,38.9342,100.4517,
188,Libohovë,AL,40.0311,20.2631,
189,Lata,PT,40.1629,-8.3327,
195,Erzin,TR,36.9559,36.2000,
207,Estrela,BR,-28.0611,-50.9383,
216,Mtimbira,TZ,-8.7833,36.3500,
223,Caravelas,BR,-17.7125,-39.2481,


In [7]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [8]:
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Keys for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']

    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("No Room in the Inn. Hit the Road.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    time.sleep(1)

# Print end of search once searching is completed
print("-------End of Search. Have a Nice Stay!-------")

Retrieving Keys for Index 108: Hambantota.
No Room in the Inn. Hit the Road.
------------
Retrieving Keys for Index 177: Zhangye.
Closest hotel in Zhangye is Huachen International Hotel Zhangye.
------------
Retrieving Keys for Index 188: Libohovë.
No Room in the Inn. Hit the Road.
------------
Retrieving Keys for Index 189: Lata.
No Room in the Inn. Hit the Road.
------------
Retrieving Keys for Index 195: Erzin.
Closest hotel in Erzin is Hattusa Vacation Thermal Club Erzin.
------------
Retrieving Keys for Index 207: Estrela.
No Room in the Inn. Hit the Road.
------------
Retrieving Keys for Index 216: Mtimbira.
No Room in the Inn. Hit the Road.
------------
Retrieving Keys for Index 223: Caravelas.
Closest hotel in Caravelas is Pousada Encanto Abrolhos.
------------
-------End of Search. Have a Nice Stay!-------


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer content
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display Map
fig


Figure(layout=FigureLayout(height='420px'))